In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # Importar Seaborn para estilos adicionales
from glob import glob
import os

In [3]:
def genes_vecinos_muc(df_genes):
    genes_vecinos_todos = {}

    # Función interna para calcular los vecinos de un gen específico
    def calcular_vecinos_muc(gen_info, df_genes):
        vecinos = []
        inicio = gen_info['start']
        fin = gen_info['end']
        strand = gen_info['strand']
        scaffold = gen_info['scaffold']
        for _, gen_info2 in df_genes.iterrows():
            if (gen_info2['strand'] != strand) and (gen_info2['scaffold'] == scaffold):
                if strand == "+":
                    distancia = abs(inicio - gen_info2['end'])
                else:
                    distancia = abs(gen_info2['start'] - fin)

                if distancia < 500:
                    vecinos.append(gen_info2['ID'])

        return vecinos

    # Iterar sobre todos los gen ID en el DataFrame
    for _, gen_info in df_genes.iterrows():
        vecinos = calcular_vecinos_muc(gen_info, df_genes)
        if vecinos:
            genes_vecinos_todos[gen_info['ID']] = vecinos

    return genes_vecinos_todos

In [ ]:
# Obtener una lista de los archivos clusters_ en el directorio actual
files_clusters = glob("clusters_*")

for file in files_clusters:
    try:
        # Obtener el nombre base del archivo clusters_
        nombre_base_clusters = os.path.splitext(file)[0][len("clusters_"):]

        # Leer el archivo de clusters
        clusters_df = pd.read_csv(file, sep=",", header=None)
        clusters_df.columns = ["Cluster", "ID"]

        # Construir el nombre del archivo .gff correspondiente
        file_gff = f"{nombre_base_clusters}_gff.csv"
        print(file_gff)

        # Verificar si el archivo .gff existe
        if os.path.exists(file_gff):
            # Leer el archivo .gff
            gff_df = pd.read_csv(file_gff, sep=",")
            
            # Merge de los archivos 
            clusters_gff = pd.merge(clusters_df, gff_df, left_on="ID", right_on="proteinId")

            
            # Mensaje de impresión
            print(f"Calculando el número de vecinos para {nombre_base_clusters}")
            # Llamada a la función 
            vecinos_dict = genes_vecinos_muc(clusters_gff)
            
            # Convertir el diccionario en un dataframe de pandas
            vecinos_dt = pd.DataFrame(list(vecinos_dict.items()), columns=['Gen_ID', 'Vecinos'])
            # Eliminar los corchetes de la columna 'Vecinos'
            vecinos_dt['Vecinos'] = vecinos_dt['Vecinos'].apply(lambda x: ', '.join(map(str, x)))

            # Guardar el dataframe en un archivo de texto
            vecinos_dt.to_csv(f"{nombre_base_clusters}_vecinos.csv", index=False)

    except Exception as e:
        print(f"Error procesando el archivo {file}: {e}")

Mucci3_gff.csv
Calculando el número de vecinos para Mucci3
Mucrac1_gff.csv
Calculando el número de vecinos para Mucrac1
Rhipu1_gff.csv
Calculando el número de vecinos para Rhipu1


In [ ]:
hhklj